# Robust Model Zoo

This notebook is used to measure robustness accuracy of the model trained for CIFAR-10 dataset. We used [AutoAttack](https://github.com/fra31/auto-attack) repository for robustness measurement.


## Install packages

In [1]:
# Install gdown for google drive download
! conda install -y gdown
# Install AutoAttack
! pip install git+https://github.com/fra31/auto-attack

In [2]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
import numpy as np
import sys
import os
from pathlib import Path
import tarfile
import shutil
import gdown

# Download required files for evaluation

Create folder structures

In [3]:
base_folder = "CIFAR-10"
dataset_folder = base_folder+"/"+"Dataset"
model_folder = base_folder+"/"+"Models/tf1"

In [4]:
if Path(base_folder).exists() == False:
    os.makedirs(base_folder)
if Path(dataset_folder).exists() == False:
    os.makedirs(dataset_folder)
if Path(model_folder).exists() == False:
    os.makedirs(model_folder)
if Path("./auto-attack").exists() == False:
    os.makedirs("./auto-attack") 

Download CIFAR-10 dataset

In [5]:
url = "https://drive.google.com/uc?export=download&id=1lt7yhOVKEVydOupZ1taA3iNXC5Ety79i"
output = dataset_folder+'/dataset_cifar10.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output, dataset_folder)
os.remove(output)

Download trained model

In [6]:
url = "https://drive.google.com/uc?export=download&id=11pqbqLtmcYgmGN5qRNbdmIFCSItHJjj_"
output = model_folder+'/checkpoint.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output, model_folder)
os.remove(output)

Download code files

In [7]:
# Download file for cifar-10 input reading
url = "https://drive.google.com/uc?export=download&id=1eOWDdFbPQ8sQJOg7ueuF-bXaL0ev-SK2"
output = base_folder+'/cifar10_input.py'
gdown.download(url, output, quiet=False)    

# Download file for TF1 model
url = "https://drive.google.com/uc?export=download&id=1bDiQHS1l5I-MDs16DcPLbHSDGHZ_jztR"
output = base_folder+'/model_submit.py'
gdown.download(url, output, quiet=False)

Download the AutoAttack clone (Commits on Jun 22, 2022)

In [8]:
url = "https://drive.google.com/uc?export=download&id=1lY3MDPYoEa29W1eCEDdUpKn60wpKdinI"
output = "./auto-attack"+'/auto-attack-TF1.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output, "./auto-attack")
os.remove(output) 

# ROBUSTNESS TEST

Modify autoattack repository after installation to run TF1 codes on TF2 backend.

Modifications on "utils_tf.py" are,
* Line 17: self.grads[cl] = tf.gradients(self.logits[:, cl], self.x_input)[0] ==> self.grads[cl] = tf.compat.v1.gradients(self.logits[:, cl], self.x_input)[0]
* Line 22: self.grad_xent = tf.gradients(self.xent, self.x_input)[0] ==> self.grad_xent = tf.compat.v1.gradients(self.xent, self.x_input)[0]
* Line 26: self.grad_dlr = tf.gradients(self.dlr, self.x_input)[0] ==> self.grad_dlr = tf.compat.v1.gradients(self.dlr, self.x_input)[0]
* Line 29: self.y_target = tf.placeholder(tf.int64, shape=[None]) ==> self.y_target = tf.compat.v1.placeholder(tf.int64, shape=[None])
* Line 32: self.grad_target = tf.gradients(self.dlr_target, self.x_input)[0] ==> self.grad_target = tf.compat.v1.gradients(self.dlr_target, self.x_input)[0]
* Line 33: self.la = tf.placeholder(tf.int64, shape=[None]) ==> self.la = tf.compat.v1.placeholder(tf.int64, shape=[None])
* Line 34: self.la_target = tf.placeholder(tf.int64, shape=[None]) ==> self.la_target = tf.compat.v1.placeholder(tf.int64, shape=[None])
* Line 40: self.grad_diff_logits = tf.gradients(self.diff_logits, self.x_input)[0] ==> self.grad_diff_logits = tf.compat.v1.gradients(self.diff_logits, self.x_input)[0]
* Line 91: tf.contrib.framework.sort(x, axis=1) ==> x_sort = tf.sort(x, axis=1)
* Line 99: tf.contrib.framework.sort(x, axis=1) ==> x_sort = tf.sort(x, axis=1)

Modifications on "autoattack.py" are,
* Line 91 and 92 are comment out since "check_dynamic" function in checks.py has not implemented to support TF1 codes. This is handled with 'the check for dynamic defenses is not currently supported' message, but model(x) function call in line 101 on checks.py is not handled for TF1 and gives error.

In [9]:
!cp "/kaggle/working/auto-attack/auto-attack-TF1/autoattack/utils_tf.py" "/opt/conda/lib/python3.7/site-packages/autoattack/utils_tf.py"
!cp "/kaggle/working/auto-attack/auto-attack-TF1/autoattack/autoattack.py" "/opt/conda/lib/python3.7/site-packages/autoattack/autoattack.py"

In [10]:
sys.path.append("./CIFAR-10")
sys.path.append("./auto-attack/auto-attack-TF1/autoattack")
from model_submit import Model
import cifar10_input
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from autoattack import AutoAttack, utils_tf

In [11]:
filter_size_list = 3
activation = "swish"
residual_sizes = 5
skip_connection_active = True
batch_normalization_active = True
class_num = 10

In [ ]:
# Define adversarial attack
num_examples = 10000
batch_size = 100
epsilon = 8/255.0

# Get model
x_input_place_holder = tf.compat.v1.placeholder(tf.float32,shape=[None, 32, 32, 3],name='x_input')
y_input_place_holder = tf.compat.v1.placeholder(tf.int64, shape=None)
model_quantize = Model(mode='eval',x_input=x_input_place_holder,y_input=y_input_place_holder,batch_size=batch_size,filter_size=filter_size_list,class_num=class_num,activation=activation,batch_normalization_active=batch_normalization_active,skip_connection_active=skip_connection_active,residual_sizes=residual_sizes,quantize_active=True)
saver = tf.compat.v1.train.Saver()

# Dataset read functions
cifar = cifar10_input.CIFAR10Data(dataset_folder)

# load data
testX = cifar.eval_data.xs[0:num_examples,:,:,:]/255.0
testY = cifar.eval_data.ys[0:num_examples]

# convert to pytorch format
torch_testX = torch.from_numpy( np.transpose(testX, (0, 3, 1, 2)) ).float().cuda()
torch_testY = torch.from_numpy( testY ).cuda()

# load model from saved weights
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# Restore the model
model_checkpoint = tf.train.latest_checkpoint(model_folder)
saver.restore(sess, model_checkpoint)

model_quantize_adapted = utils_tf.ModelAdapter(model_quantize.pre_softmax, model_quantize.x_input, model_quantize.y_input, sess)

# run plus attack
print("******** RUN PLUS ATTACK *******")
adversary = AutoAttack(model_quantize_adapted, norm='Linf', eps=epsilon, version='plus', is_tf_model=True,verbose=True)
x_adv = adversary.run_standard_evaluation(torch_testX, torch_testY, bs=batch_size)

# run attack - rand
print("******** RUN RAND ATTACK *******")
adversary = AutoAttack(model_adapted, norm='Linf', eps=epsilon, version='rand', is_tf_model=True,verbose=True)
x_adv = adversary.run_standard_evaluation(torch_testX, torch_testY, bs=batch_size)